# Summary of Kernel -> Functional Api
In this notebook we will start with a basic Functional API model
https://www.tensorflow.org/guide/keras/functional

The functional Api allows for additional functionality as opposed to the basic Sequential model which most model builders will be familiar with \
The most important features include the ability to: 
* Additional input layers  -> for multiple input data sources ( create seperate pipeline in your model for differnet datasets)
* Multiple hidden layers -> important for recurrent neural networks
* Additional outputs -> i.e. output a classification result and regression 
* Multiple metrics, losses and training sets 

For this example we will use the tabular playground competition data from [TPS February ](https://www.kaggle.com/c/tabular-playground-series-feb-2022)

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import tensorflow
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras import regularizers 

from sklearn.preprocessing import StandardScaler


In [ ]:
# experimental params 
ITERATIONS = 2000
FOLDS = 10

SCALER = StandardScaler()

PSEUDO = True
CLUSTER = True

DROP_DUPS = False

We will start with the usual processing and import of data 

# Import Data

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv", index_col = 0) 
test = pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv", index_col = 0) 
sub = pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv", index_col = 0)

In [ ]:
train.head()

In [ ]:
print( "Total segments" , len( train.columns ) -1 ) 

# Encoding 

This dataset required endoding, we will use Label Encoder from sklearn

In [ ]:
encoder = LabelEncoder()
train["target"] = encoder.fit_transform(train["target"])

# Split and Scale

In [ ]:
X = train.drop("target",axis =1)
y = train["target"]
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)

In [ ]:
scaler = SCALER
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Functional API Model

With the functional API we build each indvidual layer seperately, each layer then references the previous layer 

In [ ]:
stopping = EarlyStopping(monitor="val_loss",patience = 31)
lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=20)

In [ ]:
# build model 
input_layer = keras.Input(shape = (X.shape[1],) , name = "input_layer" ) 

# each subsequent layer will reference the previous one (see brackets at the end of each line)
hidden_1 = layers.Dense(300, activation ="selu", name = "hidden_1" )(input_layer)
hidden_2 = layers.Dense(300, activation ="selu", name = "hidden_2" )(hidden_1)

output_layer = layers.Dense(10,activation = "softmax",  name = "output_layer" )(hidden_2)

In [ ]:
# We bring this all together in our Model by referencing the input layer and final output layer
model = keras.Model(inputs = input_layer, 
                   outputs = output_layer)

In [ ]:
keras.utils.plot_model(model,show_shapes=True)

In [ ]:
model.summary()

In [ ]:
model.compile(loss= "sparse_categorical_crossentropy", metrics=["accuracy"],optimizer = "adam")
model.fit(X_train,
          y_train, 
          validation_data=(X_test,y_test), 
          batch_size=128,
          callbacks=[stopping, lr],
          epochs= ITERATIONS)

# Evaluation

In [ ]:
results = pd.DataFrame(model.history.history )
results

In [ ]:
plt.figure(figsize=(20,10))
results["val_accuracy"].plot()
results["accuracy"].plot()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
results["val_loss"].plot()
results["loss"].plot()
plt.legend()
plt.show()

In [ ]:
val_pred = model.predict(X_test)
val_pred = tensorflow.nn.softmax(val_pred).numpy()
val_pred = np.argmax(val_pred,axis =1)

print(accuracy_score(y_test,val_pred))
print(classification_report(y_test,val_pred ))

### Full Run submission 


In [ ]:
test_pred_full = model.predict(scaler.transform(test)) 

# Covert logits to probabilities 
test_pred_full = tensorflow.nn.softmax(test_pred_full).numpy()
full_preds = encoder.inverse_transform(np.argmax(test_pred_full,axis =1) )

In [ ]:
sub_full = sub.copy(deep=True)
sub_full["target"] = full_preds
sub_full.to_csv("submission_full.csv")
sub_full.head()

# Final Thoughts & bonus model

This was a quick & easy notebook to get started with the Functional API \
As you may have seen we could add many layers to our model and reference layers to others to create very complex model architectures

Below are some alternative models for this problem, however with this dataset it might not make as much sense to overcomplicate it

### Create additional features
Neural networks like homogeneous data so we will keep our features as is but create new calculated values as a seperate input

In [ ]:
original_features = test.columns

def add_feats(df):
    df["mean"] = df.mean(axis = 1)
    df["median"] = df.median(axis = 1)
    df["std"] = df.std(axis = 1)
    df["variance"] = df.std(axis = 1)
    return df

In [ ]:
add_feats(train)
add_feats(test)
train

In [ ]:
added_features = [col for col in test.columns if col not in original_features]

## Alternative model 
We will use the original features as one input and the added features as another

In [ ]:
# Create two input layers, for the original feature data and the new added features (mean, median,std etc..)
input_layer1 = layers.Input(shape=( len(original_features), ))
input_layer2 = layers.Input(shape=( len(added_features), ))

# pass inputs to seperate hidden layer
h1 = layers.Dense(300, activation = "selu")(input_layer1)
h2 = layers.Dense(300, activation = 'selu')(input_layer2)

#concatenate the hidden layers
concat_layer = layers.Concatenate(axis =1)([h1,h2])

h3 = layers.Dense(300, activation = "selu")(concat_layer)
output_layer = layers.Dense(10, activation = "softmax")(h3)

#ensure to have both inputs as a list
model = keras.Model(inputs = [input_layer1,input_layer2] , outputs = output_layer)

model.compile(loss= "sparse_categorical_crossentropy", metrics=["accuracy"],optimizer = "adam")

In [ ]:
# we need two input datasets the original features and the added (pass as a list to X)
model.fit( [ train[original_features], 
            train[added_features]],  
            y, 
          batch_size=128, 
          epochs= 10)

In [ ]:
# remember to pass two datasets for prediction
test_preds = model.predict([ test[original_features], 
                                test[added_features]]) 

In [ ]:
# Covert logits to probabilities 
test_preds = tensorflow.nn.softmax(test_preds).numpy()
final_preds = encoder.inverse_transform(np.argmax(test_preds,axis =1) )
final_preds